## 使用ASE和pyMatGen解析VASP计算结果

可以使用命令`pip3 install ase pymatgen`来安装这些第三方包

In [1]:
# 依赖包
import os,sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from ase.io.vasp import read_vasp_out
from pymatgen.io.vasp.outputs import Vasprun, Outcar
from pymatgen.io.lobster import Doscar
from pymatgen.electronic_structure.core import Spin, Orbital, OrbitalType
from pymatgen.electronic_structure.dos import DOS, CompleteDos
from pymatgen.core.structure import Site, Structure
from pymatgen.core.periodic_table import Element

In [26]:
# 解析能量
def vasp_energy(dirpath) -> float:
    if os.path.exists(os.path.join(dirpath, 'vasprun.xml')):
        out = Vasprun(os.path.join(dirpath, 'vasprun.xml'), parse_potcar_file=False)
        return float(out.final_energy)
    else:
        print('DIR: '+dirpath+" don't have OUTCAR & vasprun.xml")

# test
print('Energy of', "'vasp_example'", 'is', vasp_energy('vasp_example'), 'eV')
type(vasp_energy('vasp_example'))

/home/lgy/.local/anaconda3/lib/python3.8/site-packages/pymatgen/io/vasp/outputs.py:608: UserWarning: Final e_wo_entrp differs from the final electronic step. VASP may have included some corrections, e.g., vdw. Vasprun will return the final e_wo_entrp, i.e., including corrections in such instances.
  warnings.warn("Final e_wo_entrp differs from the final "


Energy of 'vasp_example' is -42.9856302 eV


float

In [3]:
# 解析费米能级
def vasp_efermi(dirpath) -> float:
    if os.path.exists(os.path.join(dirpath, 'vasprun.xml')):
        out = Vasprun(os.path.join(dirpath, 'vasprun.xml'), parse_potcar_file=False)
        return out.efermi
    else:
        print('DIR: '+dirpath+" don't have vasprun.xml")

# test
print('Fermi Energy of', "'vasp_example'", 'is', vasp_efermi('vasp_example'), 'eV')
type(vasp_efermi('vasp_example'))

Fermi Energy of 'vasp_example' is 1.38061341 eV


float

In [29]:
# 解析原子间距离矩阵
def vasp_distance_matrix(dirpath) -> float:
    if os.path.exists(os.path.join(dirpath, 'vasprun.xml')):
        out = Vasprun(os.path.join(dirpath, 'vasprun.xml'), parse_potcar_file=False)
        return np.around(out.structures[-1].distance_matrix, decimals=2)
    else:
        print('DIR: '+dirpath+" don't have vasprun.xml")

# test
vasp_distance_matrix('vasp_example')

<class 'numpy.ndarray'>


array([[0.  , 2.62, 2.62, 2.62, 3.  , 3.  , 3.  , 3.99, 5.62, 4.98, 5.62,
        5.62, 7.15, 7.61, 7.15, 7.15, 9.65, 9.65, 9.29, 9.65],
       [2.62, 0.  , 2.62, 2.62, 3.  , 3.  , 3.99, 3.  , 4.98, 5.62, 5.62,
        5.62, 7.61, 7.15, 7.15, 7.15, 9.65, 9.65, 9.65, 9.29],
       [2.62, 2.62, 0.  , 2.62, 3.  , 3.99, 3.  , 3.  , 5.62, 5.62, 4.98,
        5.62, 7.15, 7.15, 7.61, 7.15, 9.65, 9.28, 9.65, 9.65],
       [2.62, 2.62, 2.62, 0.  , 3.99, 3.  , 3.  , 3.  , 5.62, 5.62, 5.62,
        4.98, 7.15, 7.15, 7.15, 7.61, 9.29, 9.65, 9.65, 9.65],
       [3.  , 3.  , 3.  , 3.99, 0.  , 2.62, 2.62, 2.62, 2.82, 2.82, 2.82,
        3.85, 5.11, 5.11, 5.11, 4.39, 7.34, 6.86, 6.86, 6.86],
       [3.  , 3.  , 3.99, 3.  , 2.62, 0.  , 2.62, 2.62, 2.82, 2.82, 3.85,
        2.82, 5.11, 5.11, 4.39, 5.11, 6.86, 7.34, 6.86, 6.86],
       [3.  , 3.99, 3.  , 3.  , 2.62, 2.62, 0.  , 2.62, 3.85, 2.82, 2.82,
        2.82, 4.39, 5.11, 5.11, 5.11, 6.86, 6.86, 6.86, 7.34],
       [3.99, 3.  , 3.  , 3.  , 2.62, 2.6

In [5]:
# 解析晶格常数
def vasp_lattice_constants(dirpath):
    if os.path.exists(os.path.join(dirpath, 'vasprun.xml')):
        out = Vasprun(os.path.join(dirpath, 'vasprun.xml'), parse_potcar_file=False)
        final = out.structures[-1]
        return final.lattice.abc, final.lattice.angles
    else:
        print('DIR: '+dirpath+" don't have vasprun.xml")

vasp_lattice_constants('vasp_example')

((5.237364, 5.237364129716589, 19.91555927), (90.0, 90.0, 119.99998654849058))

In [25]:
# 解析DOS, 计算 band center
def vasp_band_center(dirpath, spd='d'):
    if spd == 'd':
        type_spd = OrbitalType.d
    elif spd == 'p':
        type_spd = OrbitalType.p
    elif spd == 's':
        type_spd = OrbitalType.s
    else:
        print('parameter "spd" error!')
        return
    if os.path.exists(os.path.join(dirpath, 'vasprun.xml')):
        out = Vasprun(os.path.join(dirpath, 'vasprun.xml'), parse_potcar_file=False)
        data = out.complete_dos
        energies = data.energies
        delta_E = np.average(energies[1:]-energies[:-1])
        dos = data.get_spd_dos()[type_spd]
        if not out.is_spin:
            densities = dos.densities
        else:
            densities = 0.5*(dos.densities[Spin.up] + dos.densities[Spin.down])
        df = pd.DataFrame()
        df['E'] = energies
        df['DOS'] = densities
        df['DOS.dE'] = densities * delta_E
        df['E.DOS.dE'] = df['E'] * df['DOS.dE']
        return np.sum(df['E.DOS.dE']) / np.sum(df['DOS.dE'])
    else:
        print('DIR: '+dirpath+" don't have vasprun.xml")
        return
    
vasp_band_center('vasp_example', spd='p')

-26.768437275685045

Help on Vasprun in module pymatgen.io.vasp.outputs object:

class Vasprun(monty.json.MSONable)
 |  Vasprun(filename, ionic_step_skip=None, ionic_step_offset=0, parse_dos=True, parse_eigen=True, parse_projected_eigen=False, parse_potcar_file=True, occu_tol=1e-08, exception_on_bad_xml=True)
 |  
 |  Vastly improved cElementTree-based parser for vasprun.xml files. Uses
 |  iterparse to support incremental parsing of large files.
 |  Speedup over Dom is at least 2x for smallish files (~1Mb) to orders of
 |  magnitude for larger files (~10Mb).
 |  
 |  **Vasp results**
 |  
 |  .. attribute:: ionic_steps
 |  
 |      All ionic steps in the run as a list of
 |      {"structure": structure at end of run,
 |      "electronic_steps": {All electronic step data in vasprun file},
 |      "stresses": stress matrix}
 |  
 |  .. attribute:: tdos
 |  
 |      Total dos calculated at the end of run.
 |  
 |  .. attribute:: idos
 |  
 |      Integrated dos calculated at the end of run.
 |  
 |  .. attri